In [2]:
%load_ext autoreload
%autoreload 2


In [21]:
import io
import json
import os
from pathlib import Path
from pprint import pprint
import requests
import sys
from typing import Optional

if '..' not in sys.path: sys.path.append('..')

from datasets import load_dataset
from datasets.arrow_dataset import Dataset
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from pydantic_yaml import parse_yaml_file_as
import torch
from torch import nn
import torch.nn.functional as F
from transformers import GPT2Tokenizer, AddedToken, PreTrainedTokenizer, AutoTokenizer


from mllm.utils.utils import read_tsv, write_tsv



# QnA datasets exploration
## Config and paths

In [12]:
DATA_PATH = Path(os.path.expandvars('$HOME')) / 'data'
HOTPOTQA_URL = 'http://curtis.ml.cmu.edu/datasets/hotpot/hotpot_train_v1.1.json'
HOTPOTQA_DATA_PATH = DATA_PATH / 'hotpotqa'

HOTPOTQA_DATA_PATH.mkdir(exist_ok=True)

## Hotpot QA

In [22]:

def get_hotpotqa(url: str, dir_path: Path) -> pd.DataFrame:
    fname = url.split('/')[-1]
    fpath = dir_path / fname
    fpath = fpath.with_suffix('.tsv')
    if fpath.exists():
        print(f'Load {fpath}')
        df = read_tsv(fpath)
    else:
        print(f'Load {url}')
        resp = requests.get(url)
        data = resp.json()
        df = pd.DataFrame.from_records(data)
        print(f'Write {fpath}')
        write_tsv(df, fpath)
    return df


In [ ]:
df_hpqa = get_hotpotqa(HOTPOTQA_URL, HOTPOTQA_DATA_PATH)
df_hpqa

Load http://curtis.ml.cmu.edu/datasets/hotpot/hotpot_train_v1.1.json
